<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas</p>

In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from src import data_processing as dp
from collections import deque

<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Ajuste Foco Camera</p>

In [ ]:
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Exposição:", cap.get(cv2.CAP_PROP_EXPOSURE))
print("Foco:", cap.get(cv2.CAP_PROP_FOCUS))


In [ ]:
def positioning(frame,frame2): 
    if np.sum(frame[:160,:160]) > 0:
        cv2.putText(frame2,                      
            'Diag. Esq',           
            (300, 470),                   
            cv2.FONT_HERSHEY_SIMPLEX,   
            1,                         
            (0, 0, 255),                
            2,                          
            cv2.LINE_AA)    
    elif np.sum(frame[:160,480:]):
        cv2.putText(frame2,                      
                'Diag. Dir',           
                (300, 470),                   
                cv2.FONT_HERSHEY_SIMPLEX,   
                1,                         
                (0, 0, 255),                
                2,                          
                cv2.LINE_AA)    
    elif np.sum(frame[:160,160:480]) > 0:
        cv2.putText(frame2,                      
            'Cima',           
            (300, 470),                   
            cv2.FONT_HERSHEY_SIMPLEX,   
            1,                         
            (0, 0, 255),                
            2,                          
            cv2.LINE_AA)   
    elif np.sum(frame[160:320,:160]) > 0:
        cv2.putText(frame2,                      
                'Esquerda',           
                (300, 470),                   
                cv2.FONT_HERSHEY_SIMPLEX,   
                1,                         
                (0, 0, 255),                
                2,                          
                cv2.LINE_AA)    
    elif np.sum(frame[160:320,480:]) > 0:
        cv2.putText(frame2,                      
                'Direita',           
                (300, 470),                   
                cv2.FONT_HERSHEY_SIMPLEX,   
                1,                         
                (0, 0, 255),                
                2,                          
                cv2.LINE_AA)


In [ ]:
cap = cv2.VideoCapture(0)

division = 16

while True:
    ret,frame_original = cap.read()
    frame_original = f.mirroring(frame_original)
    frame = frame_original.copy()
    frame = f.gray_scale(frame)
    frame = f.background_subtraction(cam_frame=frame,start_time=5,limiar=15)
    frame,vec = f.down_sampling(frame,division=division)
    frame = f.binarization(frame,limiar=5)
    vec = f.binarization(vec,limiar=5)
    original_view = frame_original.copy()
    _,vec_positioning = f.bounding_box(frame,original_view,division=division)
    positioning(frame,original_view)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        f.background_subtraction.background = f.gray_scale(frame_original.copy())
        frame_original = np.full_like(frame_original,255)

    elif cv2.waitKey(1) & 0xFF == ord('2'):
        tensor = np.array(vec)
        np.save(r'readme_media\tensor.npy', tensor)
        break

    if not ret:
        break

    # vec = vec[vec_positioning[0]:vec_positioning[1], vec_positioning[2]:vec_positioning[3]]

    cv2.imshow('Original',original_view)
    cv2.imshow('Visualizer',frame)


cap.release()
cv2.destroyAllWindows()

In [ ]:
while True:
    cam = cv2.VideoCapture(r'readme_media\wave.gif')
    while True:
        ret,frame = cam.read()
        
        if cv2.waitKey(125) & 0xFF == ord('q'):
            break
        if not ret:
            break
        frame_vizualizer = f.grid(frame)
        cv2.imshow('Vid',frame_vizualizer)
    if cv2.waitKey(500) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

In [14]:
orig = cv2.imread(r'media/image/nina.png')
bk = cv2.imread(r'media/image/nina_background_subtraction.png')
bk = bk[4:-4,648:-4]
bk = dp.gray_scale(bk)

dilated = dp.dilate(dp.binarization(bk).copy())

frames =110
for i in range(frames):
    mod = orig.copy()
    if i>=30:
        mod = dp.gray_scale(mod)
    
    if i>= 40:
        mod = bk

    if i>=50:
        mod = dp.binarization(mod,limiar=5)

    if i>=60:
        mod = dilated

    if i >=70:
        _,mod = dp.down_sampling(mod,division=8)
        mod = dp.binarization(mod)
    if i >= 80:
        _,mod = dp.bounding_box(mod,division=8)

    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    cv2.imshow('asd', mod)

cv2.destroyAllWindows()

In [46]:
import cv2
import numpy as np
import imageio



orig = cv2.imread(r'media/image/nina.png')
bk = cv2.imread(r'media/image/nina_background_subtraction.png')
bk = bk[4:-4,648:-4]
bk = dp.gray_scale(bk)

dilated = dp.dilate(dp.binarization(bk).copy())

frames = 90
gif_frames = []

for i in range(frames):
    mod = orig.copy()

    if i >= 10:
        mod = dp.gray_scale(mod)

    if i >= 20:
        mod = bk

    if i >= 30:
        mod = dp.binarization(mod, limiar=5)

    if i >= 40:
        mod = dilated

    if i >= 50:
        _, mod = dp.down_sampling(mod, division=8)
        mod = dp.binarization(mod)

    if i >= 60:
        _, mod = dp.bounding_box(mod, division=8)


    # Converte para RGB (Imageio usa RGB, OpenCV usa BGR)
    if len(mod.shape) == 2:  # escala de cinza
        mod_rgb = cv2.cvtColor(mod, cv2.COLOR_GRAY2RGB)
        mod = np.pad(mod,pad_width=((4,4),(4,4)),mode='constant',constant_values=255)

    else:
        mod_rgb = cv2.cvtColor(mod, cv2.COLOR_BGR2RGB)
        mod = np.pad(mod,pad_width=((4,4),(4,4),(0,0)),mode='constant',constant_values=255)


    gif_frames.append(mod_rgb)


    cv2.imshow('asd', mod)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# Salva como GIF (10 fps = 100 ms por frame)
imageio.mimsave('media/.bin/final_result.gif', gif_frames, duration=30,loop = 0)

In [ ]:
import imageio
from PIL import Image


frames_pil = []
for frame in frames_rgb:
    frames_pil.append(Image.fromarray(frame))


imageio.mimsave('media\.bin\gif.gif', frames_pil, duration=0.1, loop=0)
